In [22]:
matplotlib inline

In [23]:
from matplotlib import pyplot as plt
import numpy as np
import GPy

In [24]:
GPy.plotting.change_plotting_library('plotly')

In [39]:
from sklearn.model_selection import train_test_split

In [25]:
from sklearn.datasets import fetch_mldata
from IPython.display import display

In [26]:
data = fetch_mldata('mauna-loa-atmospheric-co2').data
X = data[:, [1]]
y = data[:, 0]

In [27]:
y = y.reshape(-1,1)

In [28]:
print(X.shape)
print(y.shape)

(468, 1)
(468, 1)


In [40]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [41]:
print(X_train.shape)
print(y_train.shape)

(313, 1)
(313, 1)


In [29]:
X_ = np.linspace(X.min(), X.max() + 30, 1000)[:, np.newaxis]

In [62]:
# rbf kernel, for long term trend
k1 = GPy.kern.RBF(1, np.exp(4.), np.exp(4.)) 

# periodic kernel, for seasonal component
k21 = GPy.kern.PeriodicExponential(1, variance=np.exp(1.), lengthscale=1., period=1.)
k22 = GPy.kern.RBF(1, np.exp(4.), 1.) 
k2 = k21 * k22 

# rational quadratic kernel, for fluctuations with different length scales
k3 = GPy.kern.RatQuad(1, variance=1., lengthscale=1., power=np.exp(-1.))

# rbf kernel with shorter lengthscale, month-to-month correlations
k4 = GPy.kern.RBF(1, np.exp(-2.), np.exp(-2.))

In [45]:
# rbf kernel, for long term trend
k1 = GPy.kern.RBF(1, np.exp(-2.), 1) 

# periodic kernel, for seasonal component
k21 = GPy.kern.PeriodicExponential(1, variance=np.exp(1.), lengthscale=1., period=1.)
k22 = GPy.kern.RBF(1, np.exp(-2.), 1.) 
k2 = k21 * k22 

# rational quadratic kernel, for fluctuations with different length scales
k3 = GPy.kern.RatQuad(1, variance=1., lengthscale=1., power=np.exp(-1.))

# rbf kernel with shorter lengthscale, month-to-month correlations
k4 = GPy.kern.RBF(1, np.exp(-2.), np.exp(-2.))

In [63]:
meanLinear = GPy.mappings.Linear(1,1)
#mu1.update_gradients()
meanConst = GPy.mappings.Constant(1, 1, value=-2360)

In [64]:
k = k1 + k2 + k3 + k4
#k = k1 + k3 + k4

In [65]:
mean = GPy.mappings.Additive(meanLinear, meanConst)

In [56]:
m = GPy.models.GPRegression(X_train,y_train,k,mean_function=mean)

In [66]:
m = GPy.models.GPRegression(X,y,k,mean_function=mean)

In [67]:
m.optimize_restarts(num_restarts = 5)

Optimization restart 1/5, f = 100.74169701524116
Optimization restart 2/5, f = 474.94058295904057
Optimization restart 3/5, f = 471.2938156308344
Optimization restart 4/5, f = 471.2462563669379
Optimization restart 5/5, f = 557.7777880158528


In [68]:
display(m)
fig = m.plot()
GPy.plotting.show(fig, filename='maunaloa')

GP_regression.,value,constraints,priors
mapping.linmap.A,[ 1.36628856],,
mapping.constmap.C,-2359.97417007,,
sum.rbf.variance,54.8949441526,+ve,
sum.rbf.lengthscale,53.0978735777,+ve,
sum.mul.periodic_exponential.variance,29.2324332085,+ve,
sum.mul.periodic_exponential.lengthscale,9.57793200929,+ve,
sum.mul.periodic_exponential.period,0.999954274128,+ve,
sum.mul.rbf.variance,55.3017038755,+ve,
sum.mul.rbf.lengthscale,181.127565686,+ve,
sum.RatQuad.variance,12.2700034443,+ve,


This is the format of your plot grid:
[ (1,1) x1,y1 ]

